**An Efficient Ensemble Approach for Alzheimer’s Disease
Detection Using an Adaptive Synthetic Technique and
Deep Learning**

In [4]:
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import shutil
import torch
import torch.nn.functional as F
import torchvision.models as models
from torch import nn, optim
from tqdm import tqdm

In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available. Using {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU")


GPU not available. Using CPU


<h1>Preprocessing the Dataset</h1>

In [7]:
dataset_directory = r'C:\Users\alamm\Downloads\Alzheimer Disease\archive (13)\Alzheimer_MRI_4_classes_dataset'
classes = os.listdir(dataset_directory)

In [8]:
total_images = 0
for class_name in classes:
    class_directory = os.path.join(dataset_directory, class_name)
    if os.path.isdir(class_directory):
        files = os.listdir(class_directory)
        num_files = len(files)
        print(f"Number of files in class '{class_name}': {num_files}")
        total_images += num_files

print(f"Total instances of images: {total_images}")

Number of files in class 'MildDemented': 896
Number of files in class 'ModerateDemented': 64
Number of files in class 'NonDemented': 3200
Number of files in class 'VeryMildDemented': 2240
Total instances of images: 6400


<h3>Balancing the dataset through ADASYN oversampling method</h3>

In [10]:

from torchvision import datasets, transforms
dataset = datasets.ImageFolder(root=dataset_directory)

X = [img for img, label in dataset]
y = [label for img, label in dataset]


In [11]:
import numpy as np
from imblearn.over_sampling import ADASYN
from PIL import Image

def apply_adasyn(X, y):
    X_arrays = [np.array(img) for img in X]
    
    img_shape = X_arrays[0].shape
    
    X_reshaped = np.array([x.flatten() for x in X_arrays])
    
    adasyn = ADASYN(random_state=42)
    
    X_resampled, y_resampled = adasyn.fit_resample(X_reshaped, y)

    X_balanced = [x.reshape(img_shape) for x in X_resampled]
    
    X_balanced = [Image.fromarray(x.astype('uint8')) for x in X_balanced]
    
    return X_balanced, y_resampled


In [12]:
X_balanced, y_balanced = apply_adasyn(X, y)

In [13]:
len(X_balanced)

12805

In [14]:
len(y_balanced)

12805

In [15]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 8963
Validation set size: 1281
Test set size: 2561


In [17]:
from torch.utils.data import Dataset

class AlzheimerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [18]:
from torch.utils.data import Dataset

class AlzheimerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [19]:
def train_model_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device='cuda'):
    best_val_loss = float('inf')
    patience = 5
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break
    
    return model

<h1>DenseNet121</h1>

In [21]:
import timm
import torch
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from tqdm import tqdm

model = timm.create_model("densenet121", pretrained=True, num_classes=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Loss, optimizer, scheduler setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Dataset loading
full_dataset = datasets.ImageFolder(root=dataset_directory, transform=transform)

# 80/20 split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ---- Manual Training on 1/10 of Train Set ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

max_batches = len(train_loader) // 10
num_epochs = 4

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for i, (inputs, labels) in enumerate(train_loader):
        if i >= max_batches:
            break

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(1)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")

    # Step scheduler (optional)
    scheduler.step(epoch_loss)

Epoch 1/4 - Loss: 1.1249 - Accuracy: 0.5234
Epoch 2/4 - Loss: 0.9588 - Accuracy: 0.5625


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import numpy as np

def test_model(model, test_loader, num_classes, device='cuda'):
    model.eval()
    test_correct = 0
    test_total = 0
    all_preds = []
    all_labels = []
    all_probs = []  

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)  
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.softmax(dim=1).cpu().numpy())  

    test_acc = test_correct / test_total

    report = classification_report(all_labels, all_preds,digits=4, target_names=[f'Class {i}' for i in range(num_classes)])
    
    if num_classes > 2:
        auc_score = roc_auc_score(all_labels, all_probs, multi_class='ovr')
    else:
        auc_score = roc_auc_score(all_labels, [prob[1] for prob in all_probs])

    return test_acc, report, auc_score

In [ ]:
test_loader = val_loader

test_accuracy, classification_report_str, auc_score = test_model(model, test_loader, num_classes=4, device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

<h1>Xception</h1>

In [ ]:
import timm
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm

model = timm.create_model("xception", pretrained=True, num_classes=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def train_model_with_early_stopping(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs, device, patience=5, batch_fraction=1.0):
    best_loss = float('inf')
    counter = 0
    best_model_wts = model.state_dict()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()

        running_loss = 0.0
        max_batches = int(len(train_loader) * batch_fraction)

        for i, (inputs, labels) in enumerate(train_loader):
            if i >= max_batches:
                break

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Train Loss: {running_loss / max_batches:.4f} | Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model_wts)
    return model

model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=3,
    device=device,
    batch_fraction=0.1  # <-- This trains on only 1/10th of batches
)


In [ ]:
test_loader = val_loader

test_accuracy, classification_report_str, auc_score = test_model(model, test_loader, num_classes=4, device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

<h1>EfficientNetB2</h1>

In [ ]:
import timm
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm

model = timm.create_model("efficientnet_b2", pretrained=True, num_classes=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
def train_model_with_early_stopping(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs, device, patience=5, batch_fraction=1.0):
    best_loss = float('inf')
    counter = 0
    best_model_wts = model.state_dict()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()

        running_loss = 0.0
        max_batches = int(len(train_loader) * batch_fraction)

        for i, (inputs, labels) in enumerate(train_loader):
            if i >= max_batches:
                break

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Train Loss: {running_loss / max_batches:.4f} | Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model_wts)
    return model

model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=3,
    device=device,
    batch_fraction=0.1  # <-- This trains on only 1/10th of batches
)


In [ ]:
test_loader = val_loader

test_accuracy, classification_report_str, auc_score = test_model(model, test_loader, num_classes=4, device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

<h1>VGG16</h1>

In [ ]:
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(in_features=model.classifier[6].in_features, out_features=4)    
model.to(device)

In [ ]:
def train_model_with_early_stopping(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs, device, patience=5, batch_fraction=1.0):
    best_loss = float('inf')
    counter = 0
    best_model_wts = model.state_dict()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()

        running_loss = 0.0
        max_batches = int(len(train_loader) * batch_fraction)

        for i, (inputs, labels) in enumerate(train_loader):
            if i >= max_batches:
                break

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Train Loss: {running_loss / max_batches:.4f} | Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model_wts)
    return model

model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=3,
    device=device,
    batch_fraction=0.1  # <-- This trains on only 1/10th of batches
)


In [ ]:
test_loader = val_loader

test_accuracy, classification_report_str, auc_score = test_model(model, test_loader, num_classes=4, device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

<h1>CNN</h1>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomCNN, self).__init__()
        
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batchnorm1 = nn.BatchNorm2d(32)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batchnorm2 = nn.BatchNorm2d(64)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batchnorm3 = nn.BatchNorm2d(128)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 28 * 28, 128)  # Corrected input size
        self.relu4 = nn.ReLU()
        
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # First conv block
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.batchnorm1(x)
        
        # Second conv block
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.batchnorm2(x)
        
        # Third conv block
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = self.batchnorm3(x)
        
        # Fully connected layers
        x = self.flatten(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.relu4(x)
        
        x = self.dropout2(x)
        x = self.fc2(x)
        
        # Softmax output
        x = self.softmax(x)
        
        return x

# Instantiate the model
model = CustomCNN(num_classes=4).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=50,
    device=device
)

In [ ]:
test_loader = val_loader

test_accuracy, classification_report_str, auc_score = test_model(model, test_loader, num_classes=4, device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

<h1>EfficientNet-B2+VGG16 Ensemble</h1>

In [ ]:
!pip install torchinfo
from torchinfo import summary
import timm
from torchvision import models
model = timm.create_model("efficientnet_b2", pretrained=True)
model.classifier = nn.Identity()
summary(model=model, 
        input_size=(32, 3, 224, 224),  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
model = models.vgg16(pretrained=True)
model.classifier = nn.Identity()
summary(model=model, 
        input_size=(32, 3, 224, 224),  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        # VGG16 model without classifier
        self.vgg16 = models.vgg16(pretrained=True)
        self.vgg16.classifier = nn.Identity() 
        
        # EfficientNet-B2 model without classifier
        self.efficientnet = timm.create_model("efficientnet_b2", pretrained=True)
        self.efficientnet.classifier = nn.Identity()  # Remove the classifier
        
        # Adaptive pooling to match the output sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        # Fully connected layers
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * (512 + 1408))  # Concatenating VGG16 (512) + EfficientNet-B2 (1408)
        self.fc1 = nn.Linear(7 * 7 * (512 + 1408), 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # VGG16 feature extraction
        vgg_features = self.vgg16.features(x)  # Extract features using VGG16
        vgg_features = self.adaptive_pool(vgg_features)
        
        # EfficientNet-B2 feature extraction
        efficientnet_features = self.efficientnet.forward_features(x)  # Extract features using EfficientNet-B2
        efficientnet_features = self.adaptive_pool(efficientnet_features)
        
        # Concatenate features along the channel dimension
        combined_features = torch.cat((vgg_features, efficientnet_features), dim=1)
        
        # Process combined features
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

def create_model(num_classes=4):
    return EnsembleModel(num_classes)

model = create_model(num_classes=4).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=5,
    device=device
)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
import numpy as np

def test_model(model, test_loader, num_classes, device='cuda'):
    model.eval()
    test_correct = 0
    test_total = 0
    all_preds = []
    all_labels = []
    all_probs = []  

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)  
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.softmax(dim=1).cpu().numpy())  

    test_acc = test_correct / test_total
    class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

    report = classification_report(all_labels, all_preds,digits=4, target_names= class_names)
    
    
    if num_classes > 2:
        auc_score = roc_auc_score(all_labels, all_probs, multi_class='ovr')
    else:
        auc_score = roc_auc_score(all_labels, [prob[1] for prob in all_probs])

    return test_acc, report, auc_score, all_labels, all_preds

In [ ]:
test_accuracy, classification_report_str, auc_score, true_labels, predictions = test_model(model, test_loader, num_classes=4, device='cuda')

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

cm = confusion_matrix(true_labels, predictions)
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
torch.save(model.state_dict(), "model.pt")


<h1>EfficientNet-B2+DenseNet-121 Ensemble</h1>

In [ ]:
from torchinfo import summary
import timm
from torchvision import models
model = timm.create_model("densenet121", pretrained=True)
model.classifier = nn.Identity()
summary(model=model, 
        input_size=(32, 3, 224, 224),  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        self.densenet = timm.create_model("densenet121", pretrained=True)
        self.densenet.classifier = nn.Identity() 
        
        # EfficientNet-B2 model without classifier
        self.efficientnet = timm.create_model("efficientnet_b2", pretrained=True)
        self.efficientnet.classifier = nn.Identity()  # Remove the classifier
        
        # Adaptive pooling to match the output sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        # Fully connected layers
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * (1024 + 1408))  # Concatenating VGG16 (512) + EfficientNet-B2 (1408)
        self.fc1 = nn.Linear(7 * 7 * (1024 + 1408), 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        
        densenet_features = self.densenet.forward_features(x)  # Extract features using VGG16
        densenet_features = self.adaptive_pool(densenet_features)
        
        efficientnet_features = self.efficientnet.forward_features(x)  # Extract features using EfficientNet-B2
        efficientnet_features = self.adaptive_pool(efficientnet_features)
        
        # Concatenate features along the channel dimension
        combined_features = torch.cat((densenet_features, efficientnet_features), dim=1)
        
        # Process combined features
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

def create_model(num_classes=4):
    return EnsembleModel(num_classes)

model = create_model(num_classes=4).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=5,
    device=device
)

In [ ]:
test_accuracy, classification_report_str, auc_score, true_labels, predictions = test_model(model, test_loader, num_classes=4, device='cuda')

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

cm = confusion_matrix(true_labels, predictions)
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

<h1>EfficientNet-B2+Xception</h1>

In [ ]:
from torchinfo import summary
import timm
from torchvision import models
model = timm.create_model("xception", pretrained=True)
model.fc = nn.Identity()
summary(model=model, 
        input_size=(32, 3, 224, 224),  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        self.xception = timm.create_model("xception", pretrained=True)
        self.xception.fc = nn.Identity() 
        
        self.efficientnet = timm.create_model("efficientnet_b2", pretrained=True)
        self.efficientnet.classifier = nn.Identity()  # Remove the classifier
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        # Fully connected layers
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * (2048 + 1408))  
        self.fc1 = nn.Linear(7 * 7 * (2048 + 1408), 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        
        xception_features = self.xception.forward_features(x)  
        xception_features = self.adaptive_pool(xception_features)
        
        efficientnet_features = self.efficientnet.forward_features(x)  
        efficientnet_features = self.adaptive_pool(efficientnet_features)
        
        combined_features = torch.cat((xception_features, efficientnet_features), dim=1)
        
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

def create_model(num_classes=4):
    return EnsembleModel(num_classes)

model = create_model(num_classes=4).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=50,
    device=device
)

In [ ]:
test_accuracy, classification_report_str, auc_score, true_labels, predictions = test_model(model, test_loader, num_classes=4, device='cuda')

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

cm = confusion_matrix(true_labels, predictions)
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

<h1>VGG16+DenseNet-121</h1>

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        self.vgg16 = models.vgg16(pretrained=True)
        self.vgg16.classifier = nn.Identity() 
        
        self.densenet = timm.create_model("densenet121", pretrained=True)
        self.densenet.classifier = nn.Identity() 
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * (512 + 1024))  
        self.fc1 = nn.Linear(7 * 7 * (512 + 1024), 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # VGG16 feature extraction
        vgg_features = self.vgg16.features(x)  
        vgg_features = self.adaptive_pool(vgg_features)
        
        densenet_features = self.densenet.forward_features(x)  
        densenet_features = self.adaptive_pool(densenet_features)
        
        combined_features = torch.cat((vgg_features, densenet_features), dim=1)
        
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

def create_model(num_classes=4):
    return EnsembleModel(num_classes)

model = create_model(num_classes=4).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=50,
    device=device
)

In [ ]:
test_accuracy, classification_report_str, auc_score, true_labels, predictions = test_model(model, test_loader, num_classes=4, device='cuda')

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

cm = confusion_matrix(true_labels, predictions)
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

<h1>Xception+DenseNet-121</h1>

In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision import models

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        self.xception = timm.create_model("xception", pretrained=True)
        self.xception.fc = nn.Identity() 
        
        self.densenet = timm.create_model("densenet121", pretrained=True)
        self.densenet.classifier = nn.Identity()
        
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        # Fully connected layers
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * (2048 + 1024))  
        self.fc1 = nn.Linear(7 * 7 * (2048 + 1024), 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        
        xception_features = self.xception.forward_features(x)  
        xception_features = self.adaptive_pool(xception_features)
        
        densenet_features = self.densenet.forward_features(x)  
        densenet_features = self.adaptive_pool(densenet_features)
        
        combined_features = torch.cat((xception_features, densenet_features), dim=1)
        
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

def create_model(num_classes=4):
    return EnsembleModel(num_classes)

model = create_model(num_classes=4).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
model = train_model_with_early_stopping(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=50,
    device=device
)

In [ ]:
test_accuracy, classification_report_str, auc_score, true_labels, predictions = test_model(model, test_loader, num_classes=4, device='cuda')

print(f'Test Accuracy: {test_accuracy:.4f}')
print('Classification Report:')
print(classification_report_str)
print(f'AUC Score: {auc_score:.4f}')

cm = confusion_matrix(true_labels, predictions)
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()